 # Cookies Datathon 

fksdflskdjglfdjgkldfjgkldfjlsgjdfklg

In [17]:
# for DataFrames
import numpy as np
import pandas as pd


# for graphing
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns


# for stats + stat testing
import scipy as sp
from scipy import stats 


In [18]:
# setting sns and matloplib styles
sns.set_style("whitegrid")
sns.set_palette("coolwarm")
sns.set_context("notebook", font_scale=1,  rc={"lines.linewidth": 1.5})

In [19]:
df = pd.read_csv("../Data/cookies.csv")

#### Basic data info 

In [12]:
df.head()

,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,quality,butter type,weight,diameter,mixins,crunch factor,aesthetic appeal
0,0.25,9.5,300,15.0,136.0,0.99367,8.10,0.44,12.1,8,melted,15.2,7,raisins,1.30,3
1,0.23,3.3,520,34.0,113.0,0.99429,8.16,0.48,8.4,7,melted,12.4,7,raisins,1.71,3
2,0.18,1.9,360,33.0,106.0,0.98746,8.21,0.83,14.0,9,melted,9.4,7,"nuts, chocolate",1.78,3
3,0.18,10.5,490,41.0,124.0,0.99630,8.14,0.35,10.5,7,melted,12.2,7,chocolate,1.59,3
4,0.24,2.4,770,6.0,33.0,0.99740,8.09,0.57,9.4,5,cubed,19.8,7,"nuts, oats, chocolate",1.30,3


In [14]:
df.columns

Index(['sugar to flour ratio', 'sugar index', 'bake temp', 'chill time',
       'calories', 'density', 'pH', 'grams baking soda', 'bake time',
       'quality', 'butter type', 'weight', 'diameter', 'mixins',
       'crunch factor', 'aesthetic appeal'],
      dtype='object')

In [23]:
df.shape

(5198, 16)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5198 entries, 0 to 5197
Data columns (total 16 columns):
sugar to flour ratio    5198 non-null float64
sugar index             5193 non-null float64
bake temp               5198 non-null int64
chill time              5198 non-null float64
calories                5198 non-null float64
density                 5198 non-null float64
pH                      5198 non-null float64
grams baking soda       5198 non-null float64
bake time               5188 non-null float64
quality                 5198 non-null int64
butter type             5198 non-null object
weight                  5198 non-null float64
diameter                5198 non-null int64
mixins                  5196 non-null object
crunch factor           5198 non-null float64
aesthetic appeal        5198 non-null int64
dtypes: float64(10), int64(4), object(2)
memory usage: 649.8+ KB


##### Initial statistics overview

In [6]:
df.describe()
# target is quality: index between (we think 0 and 10) but data specifies 3 to 11 - this will be reviewed later
# aesthetic appeal seems to be either 3 or 4 - we will look into this for more info
# crunch is between 1 and 2 - float (not cathegorical value)

,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,quality,weight,diameter,crunch factor,aesthetic appeal
count,5198.000000,5193.000000,5198.000000,5198.000000,5198.000000,5198.000000,5198.000000,5198.000000,5188.000000,5198.000000,5198.000000,5198.0,5198.000000,5198.000000
mean,0.318049,5.402465,559.638322,30.390246,115.015294,0.995819,8.222020,0.530864,10.494758,7.325125,14.381935,7.0,1.499367,3.000577
std,0.150036,4.668342,353.274062,17.268403,56.506171,0.062193,0.283323,0.150886,1.194584,1.302780,3.023740,0.0,0.289205,0.024019
min,0.000000,0.600000,90.000000,0.000000,-99.000000,0.987110,7.720000,0.220000,8.000000,3.000000,-99.000000,7.0,1.000000,3.000000
25%,0.240000,1.800000,380.000000,17.000000,76.000000,0.992300,8.110000,0.430000,9.500000,7.000000,12.800000,7.0,1.250000,3.000000
50%,0.310000,3.000000,470.000000,29.000000,118.000000,0.994800,8.210000,0.505000,10.300000,8.000000,14.000000,7.0,1.500000,3.000000
75%,0.390000,8.000000,640.000000,41.000000,155.000000,0.996908,8.320000,0.600000,11.300000,8.000000,15.400000,7.0,1.750000,3.000000
max,3.000000,31.600000,6110.000000,146.500000,366.500000,5.000000,25.000000,2.000000,14.900000,11.000000,31.800000,7.0,2.000000,4.000000


##### Checking NaNs

In [25]:
# look at proportion on NAN to data
df.isnull().sum()
df.isnull().sum()/len(df)*100 

sugar to flour ratio     0
sugar index              5
bake temp                0
chill time               0
calories                 0
density                  0
pH                       0
grams baking soda        0
bake time               10
quality                  0
butter type              0
weight                   0
diameter                 0
mixins                   2
crunch factor            0
aesthetic appeal         0
dtype: int64

In [33]:
# As we can have cookies with no mixins, we will not drop that type
# Instead we will replace the nan with a "No mixins" label
print(df["mixins"].unique())
df["mixins"].fillna("No mixins", inplace=True)

['raisins' 'nuts, chocolate' 'chocolate' 'nuts, oats, chocolate'
 'chocolate, oats' 'raisins, oats' 'nuts,raisins' 'nuts, oats'
 'chocolate, peanut butter' 'chocolate, oats, peanut butter'
 'peanut butter' nan 'peanut butter, raisins' 'oats']


In [35]:
# for the sugar index nans and the temperature nans
# as the proportion of data is so low, we will jsut drop them 
# Cookies with no bake time are not useful as bake time is a very important factor to the cookie 
# we will therefore drop these out
df.dropna(inplace=True)

##### Checking correlations and distributions

In [ ]:
sns.pairplot(df)

In [ ]:
sns.heatmap(df.corr())